# Create a Spreadsheet of Sonic Masking Estimation

Goal: get some sense of the level of sonic masking accross files (as it affects detection probability)
Created: 2/23/2024
Last Edited: 2/23/2024

Current status: 

Need to reduce down to just get the relevant info wanted

Idea: Look at total background dB as well as only anthropogenic noise/wind below 500 Hz

In [6]:
# Import packages
import warnings   
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import librosa
import numpy as np
import time
import subprocess
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
import IPython.display as ipd

In [7]:
# Change these to specify which year and organization you would like to process
year = "1999"
collaborator = "FWPR6"
# Can specify a specific folder to run
folder = None

In [8]:
# Settings the warnings to be ignored 
#warnings.simplefilter('ignore') 
# Create function that extracts the date and time from the file name
def extract_date_time_from_filename(filename):
    # Use a regular expression to extract the date and time from the filename
    match = re.match(r'.*_(\d{8})_(\d{6})\.wav', filename, re.IGNORECASE)
    if match:
        date_orig = match.group(1)
        # Rearrange the date to MMDDYYYY format
        date = f"{date_orig[4:6]}{date_orig[6:]}{date_orig[:4]}"
        time = match.group(2)
        return date, time
    else:
        return None, None

# Create function that calculates the average dB of three clips from each audio file
def calculate_average_db(file_path, low_freq = 200, high_freq = 3500):
    try:
        warnings.simplefilter('ignore') 
        # Load the audio file
        audio_data = Audio.from_file(file_path)
        # Apply a bandpass filter to the audio data
        filtered_audio_data = audio_data.bandpass(low_freq, high_freq, order = 10)
        # Create a numpy array
        #audio_array = np.array(filtered_audio_data.samples)
        # Calculate the decibels
        #file_db = librosa.amplitude_to_db(np.abs(audio_array))
        # Calculate the decibels directly from the filtered audio data
        file_db = librosa.amplitude_to_db(np.abs(filtered_audio_data.samples))
        average_db = np.mean(file_db)
        return average_db
    except Exception as e:
        print(f"Error calculating average dB for {file_path}: {e}")
        return np.nan


In [9]:
# Create a function to combine everything and create a datasheet 
def create_datasheet(organization_folder, output_folder):
    # Create or open a datasheet file for writing
    datasheet_path = os.path.join(output_folder, f"{year}_{collaborator}_Background_Noise.csv")
    # Open the datasheet in append mode
    with open(datasheet_path, 'a') as datasheet:
        
        # Check if the file is empty
        is_empty = os.path.getsize(datasheet_path) == 0
        # If the file is empty, write the header
        if is_empty:
            datasheet.write("File_Name,Date,Time,Datetime,Average_dB\n")

        # Iterate through each folder in the organization's data
        for site_folder in os.listdir(organization_data_folder):
            # Establish the file path for each site 
            site_folder_path = os.path.join(organization_data_folder, site_folder)
            print("site folder path:",site_folder_path)
            
            # Check if the item is a directory and not a file
            if os.path.isdir(site_folder_path):
                # Iterate through each file in the site folder
                #print("Looping through files in site folder")
                
                for filename in os.listdir(site_folder_path):
                    #print("current file name is:",filename)

                    if filename.lower().endswith(".wav"):  # Check if the file has a .wav extension
                        # Create a list of the files to omit
                        list_files_to_omit = files_to_omit["File_Name"].str.strip()
                        # Create a new file path
                        file_path = os.path.join(site_folder_path, filename)
                        # Extract date and time from the filename
                        date, time = extract_date_time_from_filename(filename)
                        # Convert date to an integer for comparison
                        date_int = int(date)
                        # Create a datetime string
                        datetime_str = f"{date} {time}"
                        # Create values for the starting date and the ending date for our target range
                        start_date = int(f"0601{year}")
                        end_date = int(f"0815{year}")
                        
                        # Convert to datetime object
                        try:
                            datetime_obj = pd.to_datetime(datetime_str, format='%m%d%Y %H%M%S')
                        except Exception as e:
                            print(f"Error in file {filename}: {e}")
                            continue  # Skip to the next iteration if there's an error
                            
                        # Check if it is in the incorrect size files and if it is, skip this iteration of the loop  
                        if filename.strip() in list_files_to_omit.values:
                            #print("Current acoustic file in list of files with incorrect size")
                            # Assign NA to the column
                            avg_db = np.nan
                        # Test if the date is in the correct date range
                        elif (date_int < start_date) or (date_int > end_date): 
                            #print("Current acoustic file outside of target date range")
                            avg_db = np.nan
                        else: 
                            # Calculate average dB
                            avg_db = calculate_average_db(file_path)

                        # Write file name and size to the datasheet
                        datasheet.write(f"{filename},{date},{time},{datetime_obj},{avg_db:.2f}\n")

    print(f"Datasheet for {year} {collaborator} created successfully!")

In [10]:
# Call the folder that has the acoustic data in it
if folder is not None:
    # For checking specific folders:
    organization_data_folder = f"F:\\Cuckoo_Acoustic_Data\\{year}\\{year}_{collaborator}_Data\\{year}_{collaborator}_Audio\\{folder}"
else:
    organization_data_folder = f"F:\\Cuckoo_Acoustic_Data\\{year}\\{year}_{collaborator}_Data\\{year}_{collaborator}_Audio"
    # For testing code
    #organization_data_folder = f"F:\\Cuckoo_Acoustic_Data\\{year}\\Test_Files\\{year}_{collaborator}_Test\\{year}_{collaborator}_AudioSubset"
    # R5 is weird and has different file sizes - looks like they had a different configuration during deployment to record 24 kHz, not 48 kHz

# Specify the output folder path
# If testing, add on test folder path
output_folder = 'C:/Users/ak201255/Documents/Cuckoo-Research/Data/Detection_Covariates'

# Read in the file for the incorrect file sizes
files_to_omit = pd.read_csv(f"F:\\CNN_Classifier_Files\\Model_2.0\\Model_Inputs_Data_Quality\\{year}_{collaborator}_IncorrectSizeAcousticFiles.csv")

# Record the start time
start_time = time.time()

# Call the function to process the organization data and generate datasheets for each site
# Commented out for safety:
create_datasheet(organization_data_folder, output_folder)

# Add in threading to process the data 
#threading.Thread(target=self.create_datasheet, args=(organization_data_folder, output_folder)).start()

# Record the end time
end_time = time.time()
# Calculate the elapsed time
elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time} minutes")

site folder path: F:\Cuckoo_Acoustic_Data\1999\1999_FWPR6_Data\1999_FWPR6_Audio\CUL-1
site folder path: F:\Cuckoo_Acoustic_Data\1999\1999_FWPR6_Data\1999_FWPR6_Audio\MISO-202
site folder path: F:\Cuckoo_Acoustic_Data\1999\1999_FWPR6_Data\1999_FWPR6_Audio\MISO-204
Datasheet for 1999 FWPR6 created successfully!
Elapsed time: 0.5671568910280863 minutes


In [ ]:
### OLD CODE
'''
# Function with bandpass filter
def calculate_average_db(file_path, low_freq = 200, high_freq = 3500):
    try:
        # Load the audio file
        audio_data = Audio.from_file(file_path)
        # Apply a bandpass filter to the audio data
        filtered_audio_data = audio_data.bandpass(low_freq, high_freq, order = 60)
        #Trim out the first five seconds and calculate average decibels in this clip
        audio_clip1 = filtered_audio_data.trim(0,5)
        db_clip1 = calc_db_clip(audio_clip1)
        #Trim out the middle five seconds and calculate average decibels in this clip
        audio_clip2 = filtered_audio_data.trim(900,905)
        db_clip2 = calc_db_clip(audio_clip2)
        #Trim out the last five seconds and calculate average decibels in this clip
        audio_clip3 = filtered_audio_data.trim(1795,1800)
        db_clip3 = calc_db_clip(audio_clip3)
        # Combine them
        all_db = np.concatenate([db_clip1, db_clip2, db_clip3])
        average_db = np.mean(all_db)
        return average_db
    except Exception as e:
        print(f"Error calculating average dB for {file_path}: {e}")
        return np.nan

def calc_db_clip(clip):
    # Create a numpy array
    clip_new = np.array(clip.samples)
    # Average the decibels from this audio file
    clip_db = librosa.amplitude_to_db(np.abs(clip_new))
    return clip_db
'''
'''
Testing new function
# Test
# Record the start time
start_time = time.time()

# Call the function to process the organization data and generate datasheets for each site
test1 = calculate_average_db('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
print(test1)

# Record the end time
end_time = time.time()
# Calculate the elapsed time
elapsed_time = (end_time - start_time)
print(f"Elapsed time: {elapsed_time} seconds")
# bandpass code with 10 db order is 5.76 seconds
# bandpass with 60 dB order is 23 s


# Load the audio file
#audio_data = Audio.from_file('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
# Apply a bandpass filter to the audio data
#filtered_audio_data = audio_data.bandpass(200, 3500, order = 60)

# Test spectrogram
#ipd.display(clear=True) # Clear previously displayed audio
#ipd.display(Spectrogram.from_audio(filtered_audio_data).plot()) # Display spectrogram
'''
'''
# FUll test
test1 = calculate_average_db('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
print(test1)
# Function gave  -50.453728 for CUL-1 
# Load the audio file
audio_data = Audio.from_file('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
# Apply a bandpass filter to the audio data
filtered_audio_data = audio_data.bandpass(200, 3500, order = 60)
audio_clip1 = filtered_audio_data.trim(0,5)
# Create a numpy array
clip_new = np.array(audio_clip1.samples)
# Average the decibels from this audio file
clip_db = librosa.amplitude_to_db(np.abs(clip_new))
audio_clip2 = filtered_audio_data.trim(900,905)
# Create a numpy array
clip_new2 = np.array(audio_clip2.samples)
# Average the decibels from this audio file
clip_db2 = librosa.amplitude_to_db(np.abs(clip_new2))
audio_clip3 = filtered_audio_data.trim(1795,1800)
# Create a numpy array
clip_new3 = np.array(audio_clip3.samples)
# Average the decibels from this audio file
clip_db3 = librosa.amplitude_to_db(np.abs(clip_new3))
clips_dbcombo = np.concatenate([clip_db, clip_db2, clip_db3])
test_mean = np.mean(clips_dbcombo)
print(test_mean)
'''
'''
test1 = calculate_average_db('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
print(test1)
#test2 = calculate_average_db('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/MISO-202/MISO-202_20230731_010000.WAV')

# Load the audio file
audio_data = Audio.from_file('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
# Apply a bandpass filter to the audio data
filtered_audio_data = audio_data.bandpass(200, 3500, order = 60)
audio_clip1 = filtered_audio_data.trim(0,5)
# Create a numpy array
clip_new = np.array(audio_clip1.samples)
# Average the decibels from this audio file
clip_db = librosa.amplitude_to_db(np.abs(clip_new))
print(type(clip_db))
print(clip_db)
audio_clip2 = filtered_audio_data.trim(900,905)
# Create a numpy array
clip_new2 = np.array(audio_clip2.samples)
# Average the decibels from this audio file
clip_db2 = librosa.amplitude_to_db(np.abs(clip_new2))
clips_dbcombo = np.concatenate([clip_db, clip_db2])
print("combined clips:", clips_dbcombo)
# Test spectrogram
#ipd.display(clear=True) # Clear previously displayed audio
#ipd.display(Spectrogram.from_audio(audio_clip1).plot()) # Display spectrogram
'''
# test 1 is CUL-1, test 2 is MISO-202
'''
# Create a function to calculate the average dB for each file
def calculate_average_db(file_path, low_freq = 1, high_freq = 4000):
    try:
        # Load the audio file
        audio_data = Audio.from_file(file_path)
        # Apply a bandpass filter to the audio data
        filtered_audio_data = audio_data.bandpass(low_freq, high_freq, order = 60)
        # Convert this to a librosa object
        audio_data_new = np.array(filtered_audio_data.samples)
        # Average the decibels from this audio file
        audio_db = librosa.amplitude_to_db(np.abs(audio_data_new))
        # Take the mean for the audio file
        average_db = np.mean(audio_db)
        return average_db
    except Exception as e:
        print(f"Error calculating average dB for {file_path}: {e}")
        return np.nan
# Test this function
test1 = calculate_average_db('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
test2 = calculate_average_db('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/MISO-202/MISO-202_20230731_010000.WAV')
#print("test 1 is:", test1)
#print("test2 is:", test2)
test1.save('C:/Users/ak201255/Documents/Testing_BandPass_Code/CUL-1_Test.WAV')
test2.save('C:/Users/ak201255/Documents/Testing_BandPass_Code/MISO-202_Test.WAV')
'''
# Check against old code
def calculate_average_db_old(file_path):
    try:
        # Load the audio file
        audio_data, _ = librosa.load(file_path)
        # Average the decibels from this audio file
        audio_db = librosa.amplitude_to_db(np.abs(audio_data))
        average_db = np.mean(audio_db)
        return average_db
    except Exception as e:
        print(f"Error calculating average dB for {file_path}: {e}")
        return np.nan
test1_old = calculate_average_db_old('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/CUL-1/CUL-1_20230601_090000.WAV')
#test2_old = calculate_average_db_old('F:/Cuckoo_Acoustic_Data/2023/Test_Files/2023_FWPR6_Test/2023_FWPR6_AudioSubset/MISO-202/MISO-202_20230731_010000.WAV')
print("test 1 is:", test1_old)
#print("test2 is:", test2_old)
'''

# Create a function to calculate the file size
def calculate_file_size(file_path):
    # Get file size in bytes
    try:
        file_size = os.path.getsize(file_path)
        # Convert to KB
        file_size_kb = file_size / 1000
        return file_size_kb
    except Exception as e:
        print(f"Error getting file size for {file_path}: {e}")
        file_size_kb = np.nan   
        return file_size_kb
#f"C:/Users/ak201255/Documents/Cuckoo-Research/Data/Acoustic_Data_Quality/Outputs/{year}/{collaborator}/Data_Quality_Sheets"
# For cleaning up 2023 data
#output_folder = f"C:/Users/ak201255/Documents/Cuckoo-Research/Data/Acoustic_Data_Quality/Outputs/{year}/{collaborator}/Data_Quality_Sheets_New"

                
#file_size_kb = calculate_file_size(file_path)

# Check if the file size matches the correct size
# for FWPR5 2022 only 86400 , 86403
#if 86400 <= file_size_kb <= 86403:
# for the rest of the data
#if 172800 <= file_size_kb <= 172802:
#    file_correct_size = "Y"
#else:
#    file_correct_size = "N"


# Create a function to combine everything and create a datasheet 
def create_datasheet(site_folder, output_folder):
    # Create or open a datasheet file for writing
    datasheet_path = os.path.join(output_folder, f"{os.path.basename(site_folder)}_Background_Noise.csv")
    # Open the datasheet in append mode
    with open(datasheet_path, 'w') as datasheet:
        # Write header to the datasheet
        datasheet.write("File_Name,Date,Time,Datetime,Average_dB\n")

        # Iterate through each file in the site folder
        for filename in os.listdir(site_folder):
            if filename.lower().endswith(".wav"):  # Check if the file has a .wav extension
                file_path = os.path.join(site_folder, filename)

                 # Extract date and time from the filename
                date, time = extract_date_time_from_filename(filename)
                # Create a datetime string
                datetime_str = f"{date} {time}"

                # Convert to datetime object
                try:
                    datetime_obj = pd.to_datetime(datetime_str, format='%m%d%Y %H%M%S')
                except Exception as e:
                    print(f"Error in file {filename}: {e}")
                    continue  # Skip to the next iteration if there's an error

                # Calculate average dB
                avg_db = calculate_average_db(file_path)

                # Write file name and size to the datasheet
                datasheet.write(f"{filename},{date},{time},{datetime_obj},{avg_db:.2f}\n")

    print(f"Datasheet for {os.path.basename(site_folder)} created successfully!")


# Create a function that creates a datasheet for each directory in a folder
def process_organization_data(organization_data_folder, output_folder):
    # Iterate through each site folder in the organization data folder
    for site_folder in os.listdir(organization_data_folder):
        # Establish the file path for each site 
        site_folder_path = os.path.join(organization_data_folder, site_folder)

        # Check if the item is a directory and not a file
        if os.path.isdir(site_folder_path):
            # Call the create_datasheet function for each site folder
            create_datasheet(site_folder_path, output_folder)
'''